In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import re
import random

df = pd.read_csv("https://raw.githubusercontent.com/riskyputh/sistem-rekomendasi/main/CSV/sistem%20rekomendasi%20list.csv")
df.head()

,NAME,ADDRESS,DESCRIPTION
0,Risky Rahmadhani Putro Utomo,Jl. Gedong Songo RT 9 RW 1 Manyaran,"Algoritma dan pemrograman, bahasa pemrograman,..."
1,PT. Sintesis Komunikasi Indonesia,"Jl. Cisanggiri V, No 6, Petogogan, Kebayoran B...",Logika dan Konsep Teknologi AI. Metode Penelit...
2,PT Mitra Semeru Indonesia,"Jl. Mampang Prapatan Raya no 84A, Tegal Parang...",Perancangan Produk/Jasa. Komunikasi. Leadershi...
3,PT Ruang Raya Indonesia,"Jl. Dr. Saharjo No.161, Manggarai Selatan, Teb...",Data & Visualization with Tableau Memahami car...
4,PT Neosia Pratama Indonusa,"Sona Topas Tower, lt. 5A/526. Jl. Jendral Sudi...",Pemodelan BIM Disiplin Struktur. Pemodelan BIM...


In [2]:
df.describe()

,NAME,ADDRESS,DESCRIPTION
count,11,11,11
unique,11,11,11
top,Risky Rahmadhani Putro Utomo,Jl. Gedong Songo RT 9 RW 1 Manyaran,"Algoritma dan pemrograman, bahasa pemrograman,..."
freq,1,1,1


**1. IKHTISAR***

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   NAME         11 non-null     object
 1   ADDRESS      11 non-null     object
 2   DESCRIPTION  11 non-null     object
dtypes: object(3)
memory usage: 392.0+ bytes


**2. Tampilan Modul Pembelajaran (Sebelum Preprocessing)**

In [9]:
def print_description(index):
    example = df[df.index == index][['NAME', 'ADDRESS','DESCRIPTION']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Name:', example[1])
        print('Address:', example[2])

In [10]:
print_description(1)

PT. Sintesis Komunikasi Indonesia
Name: Jl. Cisanggiri V, No 6, Petogogan, Kebayoran Baru, Jakarta Selatan 12170
Address: Logika dan Konsep Teknologi AI. Metode Penelitian AI. Siklus Projek AI. Pemrograman Python. Etika Profesi & Keterampilan Perusahaan. Proyek Akhir.


In [11]:
print_description(3)

PT Ruang Raya Indonesia
Name: Jl. Dr. Saharjo No.161, Manggarai Selatan, Tebet, Kota Jakarta Selatan, Daerah Khusus Ibukota Jakarta 12860
Address: Data & Visualization with Tableau Memahami cara untuk memvisualisasikan data menggunakan Tableau agar data dapat lebih mudah dipahami. Digital Product Analytics: memahami dasar-dasar digital product analytics untuk meningkatkan sistem operasional dalam bisnis. SQL for Analytics: Memahami dasar-dasar SQL dan mampu menggunakannya untuk ekstraksi data dari basis data relasional. Introduction to Data Analytics for Business: Menyusun, menyortir, memproses, dan mempelajari data bisnis menggunakan metode dan teknik tertentu untuk diubah menjadi suatu business insight. Final Project: mampu mengeksekusi suatu rangkaian projek data analitik yang mewakili real-case di industri meliputi tahap persiapan data, pengolahan data, pemodelan sampai dengan tahap visualisasi (dashboard). General Competencies: Pembelajaran tambahan yang dapat mendukung kemampuan 

In [12]:
print_description(5)

PT Nurul Fikri Cipta Inovasi
Name: Jl. Situ Indah No.116 RT. 006 RW. 010 Kel. Tugu, Kec. Cimanggis, Kota Depok, Jawa Barat
Address: Soft Skill. UI/UX. Design Pattern. Pemrograman Mobile. Code Versioning. Dasar-dasar Pemrograman. Databases. Final Project.


**3. Text Preprocessing**

In [15]:
import nltk
nltk.download('stopwords')
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z #+_]')
#stopworda = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = clean_spcl.sub(' ', text)
    text = clean_symbol.sub('', text)
    #text = ' '.join(word for word in text.split() if word not in stopworda) # hapus stopword dari kolom deskripsi
    return text
    
df['desc_clean'] = df['Description'].apply(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


KeyError: ignored

In [16]:
df.head()

,NAME,ADDRESS,DESCRIPTION
0,Risky Rahmadhani Putro Utomo,Jl. Gedong Songo RT 9 RW 1 Manyaran,"Algoritma dan pemrograman, bahasa pemrograman,..."
1,PT. Sintesis Komunikasi Indonesia,"Jl. Cisanggiri V, No 6, Petogogan, Kebayoran B...",Logika dan Konsep Teknologi AI. Metode Penelit...
2,PT Mitra Semeru Indonesia,"Jl. Mampang Prapatan Raya no 84A, Tegal Parang...",Perancangan Produk/Jasa. Komunikasi. Leadershi...
3,PT Ruang Raya Indonesia,"Jl. Dr. Saharjo No.161, Manggarai Selatan, Teb...",Data & Visualization with Tableau Memahami car...
4,PT Neosia Pratama Indonusa,"Sona Topas Tower, lt. 5A/526. Jl. Jendral Sudi...",Pemodelan BIM Disiplin Struktur. Pemodelan BIM...


**4. Tampilan Modul Pembelajaran (Setelah Preprocessing)**

In [19]:
def print_description_clean(index):
    example = df[df.index == index][['NAME', 'ADDRESS','DESCRIPTION']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Nama Perusahaan:', example[1])
        print('Alamat:', example[2])

In [20]:
print_description_clean(2)

PT Mitra Semeru Indonesia
Nama Perusahaan: Jl. Mampang Prapatan Raya no 84A, Tegal Parang, Mampang Prapatan, Jakarta Selatan
Alamat: Perancangan Produk/Jasa. Komunikasi. Leadership & Long Life Learning. Manajemen Organisasi Bisnis / Startup berbasis Teknologi Digital. Teknologi Informasi. Digital Mindset.


In [21]:
print_description_clean(4)

PT Neosia Pratama Indonusa
Nama Perusahaan: Sona Topas Tower, lt. 5A/526. Jl. Jendral Sudirman, KAV 26. Karet Kuningan, Setiabudi. Jakarta Selatan. DKI Jakarta
Alamat: Pemodelan BIM Disiplin Struktur. Pemodelan BIM Disiplin Struktur. Workflow Pemodelan Bangunan dan Infrastruktur. Presentasi dan Visualisasi Project. Workflow analisis dan kolaborasi tim pada Project. Pemodelan BIM Analisis Struktur. Pemodelan BIM Disiplin Mekanikal.Pemodelan BIM Disiplin Elektrikal. Pemodelan BIM Generative Components. Project Kolaborasi Digital AEC.


In [22]:
print_description_clean(5)

PT Nurul Fikri Cipta Inovasi
Nama Perusahaan: Jl. Situ Indah No.116 RT. 006 RW. 010 Kel. Tugu, Kec. Cimanggis, Kota Depok, Jawa Barat
Alamat: Soft Skill. UI/UX. Design Pattern. Pemrograman Mobile. Code Versioning. Dasar-dasar Pemrograman. Databases. Final Project.


**5. TF-IDF & Cosine Similarity**

In [26]:
df.set_index('NAME', inplace=True)
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['DESCRIPTION'])
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cos_sim

array([[1.        , 0.04602266, 0.02643456, 0.11512061, 0.01782708,
        0.09306197, 0.        , 0.        , 0.01662809, 0.01207077,
        0.        ],
       [0.04602266, 1.        , 0.0264492 , 0.01581042, 0.00729756,
        0.02250548, 0.        , 0.00726181, 0.00696313, 0.00423297,
        0.00660336],
       [0.02643456, 0.0264492 , 1.        , 0.02556433, 0.00750074,
        0.        , 0.        , 0.        , 0.00869002, 0.01634321,
        0.        ],
       [0.11512061, 0.01581042, 0.02556433, 1.        , 0.0340072 ,
        0.06677112, 0.00464875, 0.01548134, 0.01446452, 0.01185024,
        0.01631462],
       [0.01782708, 0.00729756, 0.00750074, 0.0340072 , 1.        ,
        0.01549884, 0.        , 0.        , 0.01343356, 0.00583023,
        0.        ],
       [0.09306197, 0.02250548, 0.        , 0.06677112, 0.01549884,
        1.        , 0.        , 0.        , 0.        , 0.00899015,
        0.00975172],
       [0.        , 0.        , 0.        , 0.00464875, 0.

In [27]:
indices = pd.Series(df.index)
indices[:50]

0               Risky Rahmadhani Putro Utomo
1          PT. Sintesis Komunikasi Indonesia
2                  PT Mitra Semeru Indonesia
3                    PT Ruang Raya Indonesia
4                 PT Neosia Pratama Indonusa
5               PT Nurul Fikri Cipta Inovasi
6     PT Bank Negara Indonesia (Persero) Tbk
7     PT Bank Rakyat Indonesia (Persero) Tbk
8        PT Abbauf Mulia Konsultan Teknologi
9                      Yayasan Hasnur Centre
10         PT Telkom Indonesia (Persero) Tbk
Name: NAME, dtype: object

**6. Modelling**

In [28]:
def recommendations(name, cos_sim = cos_sim):
    
    recommended_modul = []
    
    # Mengambil nama modul berdasarkan variabel indicies
    idx = indices[indices == name].index[0]

    # Membuat series berdasarkan skor kesamaan
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)

    # mengambil index dan dibuat 3 baris rekomendasi terbaik
    top_10_indexes = list(score_series.iloc[1:4].index)
    
    for i in top_10_indexes:
        recommended_modul.append(list(df.index)[i])
        
    return recommended_modul

**7. Prediksi**

In [29]:
recommendations('Risky Rahmadhani Putro Utomo')

['PT Ruang Raya Indonesia',
 'PT Nurul Fikri Cipta Inovasi',
 'PT. Sintesis Komunikasi Indonesia']

In [30]:
recommendations('PT. Sintesis Komunikasi Indonesia')

['Risky Rahmadhani Putro Utomo',
 'PT Mitra Semeru Indonesia',
 'PT Nurul Fikri Cipta Inovasi']

In [31]:
recommendations('PT Telkom Indonesia (Persero) Tbk')

['PT Bank Rakyat Indonesia (Persero) Tbk',
 'PT Ruang Raya Indonesia',
 'PT Nurul Fikri Cipta Inovasi']